# Programming for Data Science and Artificial Intelligence

## 14 Deep Neural Network from Scratch

### Readings

- [WEIDMAN] Ch3
- [CHARU] Ch2-3



As a recap, last time, we have inputted our data into a linear function wwhich we got some decent result.  To improve, we inserted a non-linear function in between follow by a linear function, which obviously increase the result since it help model the non-linearity.  We can summarize that neural newtork has basically the following key things that make it work:

1. **Activation function (we can also generalized as Operations)**: these functions help model input data into a non-linear relationship

2. **Chain rule / Backpropagation**: they are essential for us to improve the neural network

3. **Layers of neurons**: they are performing some sequential processes that split out desired output.

In fact, you are now very close to understanding Deep Neural Networks.  In this lesson, we have several objectives:

- From our low-level understandings of neural network, we shall code them up as a class, so they are resuable.  They will be essential for understanding deep neural network, CNN, and RNN.  You will be so surprised that all these fancy terms are simply layers after layers

- Of course, we shall also understand what is "deep" neural network.  Here, we shall simply say that "deep" neural network is simply neural network that has more than "one" hidden layers (which we did not yet define what is "hidden" layers)  